In [1]:
#NOTE: use python env acmil in ACMIL folder
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, Subset, ConcatDataset
import torch.optim as optim
from pathlib import Path
import PIL
from skimage import filters
import random

    
sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, set_seed
from Utils import log_message
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction_sepatt, BCE_Weighted_Reg, BCE_Weighted_Reg_focal, compute_loss_for_all_labels_sepatt
from Model import Mutation_MIL_MT_sepAtt #, Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline


#FOR ACMIL
current_dir = os.getcwd()
grandparent_subfolder = os.path.join(current_dir, '..', '..', 'other_model_code','ACMIL-main')
grandparent_subfolder = os.path.normpath(grandparent_subfolder)
sys.path.insert(0, grandparent_subfolder)
from utils.utils import save_model, Struct, set_seed
import yaml
import sys
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import yaml
from pprint import pprint

import argparse
import torch
from torch import nn
from torch.utils.data import DataLoader

from utils.utils import save_model, Struct, set_seed
from datasets.datasets import build_HDF5_feat_dataset
from architecture.transformer import ACMIL_GA #ACMIL_GA
from architecture.transformer import ACMIL_MHA
import torch.nn.functional as F

from utils.utils import MetricLogger, SmoothedValue, adjust_learning_rate
from timm.utils import accuracy
import torchmetrics
import wandb

In [2]:
def predict(net, criterion, data_loader, device, conf, header):

    metric_logger = MetricLogger(delimiter="  ")
    
    y_pred = []
    y_true = []
    y_pred_prob = []
    # Set the network to evaluation mode
    net.eval()
    for data in metric_logger.log_every(data_loader, 100, header):
        image_patches = data[0].to(device, dtype=torch.float32)
        label_lists = data[1][0]
        sub_preds_list, slide_preds_list, attn_list = net(image_patches) #lists len of n of tasks, each task = [5,2], [1,2], [1,5,3],
        
        #Compute loss for each task, then sum
        pred_list = []
        pred_prob_list = []
        for k in range(conf.n_task):
            sub_preds = sub_preds_list[k]
            slide_preds = slide_preds_list[k]
            attn = attn_list[k]
            labels = label_lists[:,k].to(device, dtype = torch.int64).to(device)
            pred = torch.softmax(slide_preds, dim=-1)
            pred_list.append(pred)
            pred_prob = torch.softmax(slide_preds, dim=-1)[:,1]
            pred_prob_list.append(pred_prob)
    
        y_pred.append(pred_list)
        y_true.append(label_lists)
        y_pred_prob.append(pred_prob_list)

    #Get prediction for each task
    y_predprob_task = []
    y_pred_tasks = []
    y_true_tasks = []
    for k in range(conf.n_task):
        y_pred_tasks.append([p[k] for p in y_pred])
        y_predprob_task.append([p[k].item() for p in y_pred_prob])
        y_true_tasks.append([t[:,k].to(device, dtype = torch.int64).item() for t in y_true])
    
    return y_pred_tasks, y_predprob_task, y_true_tasks

def get_performance(y_predprob, y_true, cohort_ids, outcome):

    #Prediction df
    pred_df = pd.DataFrame({"SAMPLE_IDs":  cohort_ids, 
                            "Y_True": y_true, 
                            "Pred_Prob" :  y_predprob,
                            "OUTCOME": outcome})
        
    THRES = round(pred_df['Pred_Prob'].quantile(0.8),2)
    pred_df['Pred_Class'] = 0
    pred_df.loc[pred_df['Pred_Prob'] > THRES,'Pred_Class'] = 1


    perf_df = compute_performance_each_label([outcome], pred_df, "SAMPLE_LEVEL")

    return pred_df, perf_df

In [3]:
from architecture.network import Classifier_1fc, DimReduction, DimReduction1

class Classifier_multitask(nn.Module):
    def __init__(self, n_channels, n_classes, n_tasks, droprate=0.0):
        super(Classifier_multitask, self).__init__()
        self.fc =  nn.ModuleList([nn.Linear(n_channels, n_classes) for _ in range(n_tasks)])  
        self.droprate = droprate
        if self.droprate != 0.0:
            self.dropout = torch.nn.Dropout(p=self.droprate)

    def forward(self, x):

        if self.droprate != 0.0:
            x = self.dropout(x)

        out = []
        for i in range(len(self.fc)):
            out.append(self.fc[i](x))
            
        return out
        
class Attention_Gated(nn.Module):
    def __init__(self, L=512, D=128, K=1):
        super(Attention_Gated, self).__init__()

        self.L = L
        self.D = D
        self.K = K

        self.attention_V = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh()
        )

        self.attention_U = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Sigmoid()
        )

        self.attention_weights = nn.Linear(self.D, self.K)

    def forward(self, x):
        ## x: N x L
        A_V = self.attention_V(x)  # NxD
        A_U = self.attention_U(x)  # NxD
        A = self.attention_weights(A_V * A_U) # NxK
        A = torch.transpose(A, 1, 0)  # KxN


        return A  ### K x N


class ACMIL_GA_MultiTask(nn.Module):
    def __init__(self, conf, D=128, droprate=0, n_token=1, n_masked_patch=0, mask_drop=0, n_task = 7):
        super(ACMIL_GA_MultiTask, self).__init__()
        self.dimreduction = DimReduction(conf.D_feat, conf.D_inner)
        self.attention_multitask = nn.ModuleList()
        for i in range(n_task):
            self.attention_multitask.append(Attention_Gated(conf.D_inner, D, n_token))
        
        self.classifier_multitask = nn.ModuleList()
        for i in range(n_task):
            classifier = nn.ModuleList()
            for j in range(n_token):
                classifier.append(Classifier_1fc(conf.D_inner, conf.n_class, droprate))
            self.classifier_multitask.append(classifier)
        
        self.n_masked_patch = n_masked_patch
        self.n_token = conf.n_token

        self.Slide_classifier_multitask = nn.ModuleList()
        for i in range(n_task):
            Slide_classifier = Classifier_1fc(conf.D_inner, conf.n_class, droprate)
            self.Slide_classifier_multitask.append(Slide_classifier)
            
        self.mask_drop = mask_drop


    def forward(self, x): ## x: N x L
        x = x[0]
        x = self.dimreduction(x)

        #Each task has its own attention
        A_out_list = []
        outputs_list = []
        bag_feat_list = []
        for i, head in enumerate(self.attention_multitask): 
            A = head(x)  ## K x N
            
            if self.n_masked_patch > 0 and self.training:
                # Get the indices of the top-k largest values
                k, n = A.shape
                n_masked_patch = min(self.n_masked_patch, n)
                _, indices = torch.topk(A, n_masked_patch, dim=-1)
                rand_selected = torch.argsort(torch.rand(*indices.shape), dim=-1)[:,:int(n_masked_patch * self.mask_drop)]
                masked_indices = indices[torch.arange(indices.shape[0]).unsqueeze(-1), rand_selected]
                random_mask = torch.ones(k, n).to(A.device)
                random_mask.scatter_(-1, masked_indices, 0)
                A = A.masked_fill(random_mask == 0, -1e9)
            
            A_out = A
            A_out_list.append(A_out.unsqueeze(0))
            
            A = F.softmax(A, dim=1)  # softmax over N
            afeat = torch.mm(A, x) ## K x L
            outputs = []
            for j, head2 in enumerate(self.classifier_multitask[i]):
                outputs.append(head2(afeat[j]))
            outputs_list.append(torch.stack(outputs, dim=0))
            
            bag_A = F.softmax(A_out, dim=1).mean(0, keepdim=True)
            bag_feat = torch.mm(bag_A, x)
            bag_feat_list.append(self.Slide_classifier_multitask[i](bag_feat))
            
        return outputs_list, bag_feat_list, A_out_list #torch.stack(outputs, dim=0), self.Slide_classifier(bag_feat), A_out.unsqueeze(0) #torch.stack(outputs, dim=0)

    def forward_feature(self, x, use_attention_mask=False): ## x: N x L
        x = x[0]
        x = self.dimreduction(x)
        A = self.attention(x)  ## K x N


        if self.n_masked_patch > 0 and use_attention_mask:
            # Get the indices of the top-k largest values
            k, n = A.shape
            n_masked_patch = min(self.n_masked_patch, n)
            _, indices = torch.topk(A, n_masked_patch, dim=-1)
            rand_selected = torch.argsort(torch.rand(*indices.shape), dim=-1)[:,:int(n_masked_patch * self.mask_drop)]
            masked_indices = indices[torch.arange(indices.shape[0]).unsqueeze(-1), rand_selected]
            random_mask = torch.ones(k, n).to(A.device)
            random_mask.scatter_(-1, masked_indices, 0)
            A = A.masked_fill(random_mask == 0, -1e9)

        A_out = A
        bag_A = F.softmax(A_out, dim=1).mean(0, keepdim=True)
        bag_feat = torch.mm(bag_A, x)
        return bag_feat

In [4]:
####################################
######      USERINPUT       ########
####################################
ALL_LABELS = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
TUMOR_FRAC_THRES = 0.9 
feature_extraction_method = 'uni2' #retccl, uni1
learning_method = "acmil"
focal_gamma = 2
focal_alpha = 0.1
loss_method = 'REGLOSS' #ATTLOSS

################################
#model Para
BATCH_SIZE  = 1
DROPOUT = 0
DIM_OUT = 128
SELECTED_MUTATION = "MT"
SELECTED_FOLD = 0
arch = 'ga_mt' #ga_mt or ga

if feature_extraction_method == 'retccl':
    SELECTED_FEATURE = [str(i) for i in range(0,2048)] + ['TUMOR_PIXEL_PERC'] #If retccl 2048, if uni 1024
    N_FEATURE = 2048
elif feature_extraction_method == 'uni1': 
    SELECTED_FEATURE = [str(i) for i in range(0,1024)] + ['TUMOR_PIXEL_PERC'] #If retccl 2048, if uni 1024
    N_FEATURE = 1024
elif feature_extraction_method == 'uni2':
    SELECTED_FEATURE = [str(i) for i in range(0,1536)] + ['TUMOR_PIXEL_PERC'] #If retccl 2048, if uni 1024
    N_FEATURE = 1536
    
################################
# get config
config_dir = "myconf.yml"
with open(config_dir, "r") as ymlfile:
    c = yaml.load(ymlfile, Loader=yaml.FullLoader)
    #c.update(vars(args))
    conf = Struct(**c)

conf.train_epoch = 100
conf.D_feat = N_FEATURE
conf.D_inner = DIM_OUT
conf.n_token = 3
conf.n_class = 2
conf.wandb_mode = 'disabled'
conf.mask_drop = 0.6
conf.n_masked_patch = 0
conf.n_task = 7
#conf.lr = 0.000001 #change this for HR only

# Print all key-value pairs in the conf object
for key, value in conf.__dict__.items():
    print(f"{key}: {value}")
    
##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
folder_name_overlap = "IMSIZE250_OL100"
folder_name_nonoverlap = "IMSIZE250_OL0"
feature_path_opx_train =  os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', "OPX", folder_name_overlap, 'feature_' + feature_extraction_method, 'TFT' + str(TUMOR_FRAC_THRES))
feature_path_opx_test =  os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', "OPX", folder_name_nonoverlap, 'feature_' + feature_extraction_method, 'TFT' + str(TUMOR_FRAC_THRES))
feature_path_tma = os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', "TAN_TMA_Cores",folder_name_nonoverlap, 'feature_' + feature_extraction_method, 'TFT' + str(TUMOR_FRAC_THRES))
folder_name_ids = 'uni1/TrainOL100_TestOL0_TFT' + str(TUMOR_FRAC_THRES)  + "/"
train_val_test_id_path =  os.path.join(proj_dir + 'intermediate_data/6_Train_TEST_IDS', folder_name_ids)

train_epoch: 100
warmup_epoch: 0
wd: 1e-05
lr: 0.0001
min_lr: 0
dataset: bracs
B: 1
n_class: 2
n_worker: 8
pin_memory: False
n_shot: -1
D_feat: 1536
D_inner: 128
n_token: 3
wandb_mode: disabled
mask_drop: 0.6
n_masked_patch: 0
n_task: 7


In [5]:
######################
#Create output-dir
################################################
folder_name1 = feature_extraction_method + '/TrainOL100_TestOL0_TFT' + str(TUMOR_FRAC_THRES)  + "/"
outdir0 =  proj_dir + "intermediate_data/pred_out02122025" + "/" + folder_name1 + 'FOLD' + str(SELECTED_FOLD) + '/' + SELECTED_MUTATION + "/" 
outdir1 =  outdir0  + "/saved_model/"
outdir2 =  outdir0  + "/model_para/"
outdir3 =  outdir0  + "/logs/"
outdir4 =  outdir0  + "/predictions/"
outdir5 =  outdir0  + "/perf/"


create_dir_if_not_exists(outdir0)
create_dir_if_not_exists(outdir1)
create_dir_if_not_exists(outdir2)
create_dir_if_not_exists(outdir3)
create_dir_if_not_exists(outdir4)
create_dir_if_not_exists(outdir5)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//saved_model/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//model_para/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//logs/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//predictions/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//perf/' already exists.
cuda:0


In [6]:
################################################
#     Model ready data 
################################################
opx_data_ol100 = torch.load(feature_path_opx_train + '/OPX_data.pth')
opx_ids_ol100 = torch.load(feature_path_opx_train + '/OPX_ids.pth')
opx_info_ol100  = torch.load(feature_path_opx_train + '/OPX_info.pth')

opx_data_ol0 = torch.load(feature_path_opx_test + '/OPX_data.pth')
opx_ids_ol0 = torch.load(feature_path_opx_test + '/OPX_ids.pth')
opx_info_ol0  = torch.load(feature_path_opx_test + '/OPX_info.pth')

tma_data = torch.load(feature_path_tma + '/tma_data.pth')
tma_ids = torch.load(feature_path_tma + '/tma_ids.pth')
tma_info  = torch.load(feature_path_tma + '/tma_info.pth')

In [7]:
########################################################
#Update tma
########################################################
haslabel_indexes = []
for i in range(len(tma_data)):
    if torch.isnan(tma_data[i][1]).all() == False:
        #print(f"Item {i} has the second element all NaNs.")
        haslabel_indexes.append(i)


tma_data = Subset(tma_data, haslabel_indexes)
tma_ids = list(Subset(tma_ids, haslabel_indexes))
tma_info = list(Subset(tma_info, haslabel_indexes))
len(tma_info) #355 if TF0.9, a lot of cores does not have enough cancer tiles > 0.9

355

In [8]:
################################################
#Get train, test IDs
#NOTE: this was in the old train: ['OPX_207','OPX_209','OPX_213','OPX_214','OPX_215']
################################################
train_test_val_id_df = pd.read_csv(train_val_test_id_path + "train_test_split.csv")
train_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'TRAIN', 'SAMPLE_ID'])
test_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'TEST', 'SAMPLE_ID'])
val_ids_all = list(train_test_val_id_df.loc[train_test_val_id_df['FOLD' + str(SELECTED_FOLD)] == 'VALID', 'SAMPLE_ID'])

In [9]:
################################################
#Get Train, test, val data
################################################
#Train:
inc_idx = [opx_ids_ol100.index(x) for x in train_ids_all]
train_data = Subset(opx_data_ol100, inc_idx)
train_ids =  list(Subset(opx_ids_ol100, inc_idx))
train_info = list(Subset(opx_info_ol100, inc_idx))

#Val:
inc_idx = [opx_ids_ol100.index(x) for x in val_ids_all]
val_data = Subset(opx_data_ol100, inc_idx)
val_ids =  list(Subset(opx_ids_ol100, inc_idx))
val_info = list(Subset(opx_info_ol100, inc_idx))

#Test:
inc_idx = [opx_ids_ol0.index(x) for x in test_ids_all]
test_data = Subset(opx_data_ol0, inc_idx)
test_ids =  list(Subset(opx_ids_ol0, inc_idx))
test_info = list(Subset(opx_info_ol0, inc_idx))

In [10]:
#count labels in train
train_label_counts = [dt[1] for dt in train_data]
train_label_counts = torch.concat(train_label_counts)
count_ones = (train_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/train_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers)

#count labels in test
test_label_counts = [dt[1] for dt in test_data]
test_label_counts = torch.concat(test_label_counts)
count_ones = (test_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/test_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers)

#count labels in tma
tma_label_counts = [dt[1] for dt in tma_data] 
tma_label_counts = torch.concat(tma_label_counts)
count_ones = (tma_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/tma_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers) #["AR","PTEN","RB1","TP53"

tensor([15, 25, 32, 18, 55, 10,  9])
['10.0', '16.7', '21.3', '12.0', '36.7', '6.7', '6.0']
tensor([ 5,  7,  8,  4, 18,  7,  7])
['11.1', '15.6', '17.8', '8.9', '40.0', '15.6', '15.6']
tensor([216,   0, 119, 107, 178,   0,   0])
['60.8', '0.0', '33.5', '30.1', '50.1', '0.0', '0.0']


In [11]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

150
8
45


In [12]:
####################################################
#            Train 
####################################################
set_seed(0)
#Dataloader for training
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
tma_loader = DataLoader(dataset=tma_data, batch_size=BATCH_SIZE, shuffle=False)


In [13]:
# define network
if arch == 'ga':
    model = ACMIL_GA(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop= conf.mask_drop)
elif arch == 'ga_mt':
    model = ACMIL_GA_MultiTask(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop= conf.mask_drop, n_task = conf.n_task)
else:
    model = ACMIL_MHA(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop=conf.mask_drop)
model.to(device)


class FocalLoss_withATT(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss_withATT, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.att_reg_flag = True
        self.att_reg_loss = nn.MSELoss()

    def forward(self, inputs, targets, tumor_fractions, attention_scores):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            F_loss =  F_loss.mean()
        elif self.reduction == 'sum':
            F_loss =  F_loss.sum()

        if self.att_reg_flag == True:
            attention_scores_mean = torch.softmax(attention_scores, dim=-1).mean(dim = 1) #Take the mean across all braches
            F_loss = F_loss + self.att_reg_loss(tumor_fractions, attention_scores)

        return F_loss

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss
            
# Example usage:
criterion = FocalLoss(alpha=focal_alpha, gamma=focal_gamma, reduction='mean')
#criterion = nn.CrossEntropyLoss()

# define optimizer, lr not important at this point
optimizer0 = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=conf.wd)

In [14]:
def train_one_epoch_multitask(model, criterion, data_loader, optimizer0, device, epoch, conf):
    """
    Trains the given network for one epoch according to given criterions (loss functions)
    """

    # Set the network to training mode
    model.train()

    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 100


    for data_it, data in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        # for data_it, data in enumerate(data_loader, start=epoch * len(data_loader)):
        # Move input batch onto GPU if eager execution is enabled (default), else leave it on CPU
        # Data is a dict with keys `input` (patches) and `{task_name}` (labels for given task)
        image_patches = data[0].to(device, dtype=torch.float32)
        label_lists = data[1][0]
        tf = data[2].to(device, dtype=torch.float32)

        # # Calculate and set new learning rate
        adjust_learning_rate(optimizer0, epoch + data_it/len(data_loader), conf)

        # Compute loss
        sub_preds_list, slide_preds_list, attn_list = model(image_patches) #lists len of n of tasks, each task = [5,2], [1,2], [1,5,3],
        
        #Compute loss for each task, then sum
        loss = 0
        for k in range(conf.n_task):
            sub_preds = sub_preds_list[k]
            slide_preds = slide_preds_list[k]
            attn = attn_list[k]
            labels = label_lists[:,k].to(device, dtype = torch.int64).to(device)
        
        
            
            if conf.n_token > 1:
                loss0 = criterion(sub_preds, labels.repeat_interleave(conf.n_token))
            else:
                loss0 = torch.tensor(0.)
            loss1 = criterion(slide_preds, labels)
            
            
            diff_loss = torch.tensor(0).to(device, dtype=torch.float)
            attn = torch.softmax(attn, dim=-1)
            
            for i in range(conf.n_token):
                for j in range(i + 1, conf.n_token):
                    diff_loss += torch.cosine_similarity(attn[:, i], attn[:, j], dim=-1).mean() / (
                                conf.n_token * (conf.n_token - 1) / 2)

            if loss_method == 'ATTLOSS': #ATTLOSS
                #ATT loss
                # avg_attn = attn.mean(dim = 1) #Across tokens
                # loss2 = F.mse_loss(avg_attn, tf)
                #for each token
                loss2 = 0
                for i in range(conf.n_token):
                    loss2 += F.mse_loss(attn[:,i,:], tf)
                loss += diff_loss + loss0 + loss1 + loss2
            else:
                loss += diff_loss + loss0 + loss1 

        optimizer0.zero_grad()
        # Backpropagate error and update parameters
        loss.backward()
        optimizer0.step()


        metric_logger.update(lr=optimizer0.param_groups[0]['lr'])
        metric_logger.update(sub_loss=loss0.item())
        metric_logger.update(diff_loss=diff_loss.item())
        metric_logger.update(slide_loss=loss1.item())

        if conf.wandb_mode != 'disabled':
            """ We use epoch_1000x as the x-axis in tensorboard.
            This calibrates different curves when batch size changes.
            """
            wandb.log({'sub_loss': loss0}, commit=False)
            wandb.log({'diff_loss': diff_loss}, commit=False)
            wandb.log({'slide_loss': loss1})
            #wandb.log({'att_loss': loss2})

In [15]:
# Disable gradient calculation during evaluation
@torch.no_grad()
def evaluate_multitask(net, criterion, data_loader, device, conf, header):

    # Set the network to evaluation mode
    net.eval()

    y_pred = []
    y_true = []

    metric_logger = MetricLogger(delimiter="  ")

    for data in metric_logger.log_every(data_loader, 100, header):
        image_patches = data[0].to(device, dtype=torch.float32)
        label_lists = data[1][0]
        tf = data[2].to(device, dtype=torch.float32)


        sub_preds_list, slide_preds_list, attn_list = net(image_patches) #lists len of n of tasks, each task = [5,2], [1,2], [1,5,3],
        
        #Compute loss for each task, then sum
        loss = 0
        div_loss = 0
        pred_list = []
        acc1_list = []
        for k in range(conf.n_task):
            sub_preds = sub_preds_list[k]
            slide_preds = slide_preds_list[k]
            attn = attn_list[k]
            labels = label_lists[:,k].to(device, dtype = torch.int64).to(device)
            
            div_loss += torch.sum(F.softmax(attn, dim=-1) * F.log_softmax(attn, dim=-1)) / attn.shape[1]
            loss += criterion(slide_preds, labels)
            pred = torch.softmax(slide_preds, dim=-1)
            acc1 = accuracy(pred, labels, topk=(1,))[0]

            pred_list.append(pred)
            acc1_list.append(acc1)
            
        avg_acc = sum(acc1_list)/conf.n_task

        metric_logger.update(loss=loss.item())
        metric_logger.update(div_loss=div_loss.item())
        metric_logger.meters['acc1'].update(avg_acc.item(), n=labels.shape[0])

        y_pred.append(pred_list)
        y_true.append(label_lists)

    #Get prediction for each task
    y_pred_tasks = []
    y_true_tasks = []
    for k in range(conf.n_task):
        y_pred_tasks.append([p[k] for p in y_pred])
        y_true_tasks.append([t[:,k].to(device, dtype = torch.int64) for t in y_true])
    
    #get performance for each calss
    auroc_each = 0
    f1_score_each = 0
    for k in range(conf.n_task):
        y_pred_each = torch.cat(y_pred_tasks[k], dim=0)
        y_true_each = torch.cat(y_true_tasks[k], dim=0)
    
        AUROC_metric = torchmetrics.AUROC(num_classes = conf.n_class, task='multiclass').to(device)
        AUROC_metric(y_pred_each, y_true_each)
        auroc_each += AUROC_metric.compute().item()
    
        F1_metric = torchmetrics.F1Score(num_classes = conf.n_class, task='multiclass').to(device)
        F1_metric(y_pred_each, y_true_each)
        f1_score_each += F1_metric.compute().item()
        print("AUROC",str(k),":",AUROC_metric.compute().item())
    auroc = auroc_each/conf.n_task
    f1_score = f1_score_each/conf.n_task

    print('* Acc@1 {top1.global_avg:.3f} loss {losses.global_avg:.3f} auroc {AUROC:.3f} f1_score {F1:.3f}'
          .format(top1=metric_logger.acc1, losses=metric_logger.loss, AUROC=auroc, F1=f1_score))

    return auroc, metric_logger.acc1.global_avg, f1_score, metric_logger.loss.global_avg

In [16]:
ckpt_dir = outdir1 + SELECTED_MUTATION + "/"
create_dir_if_not_exists(ckpt_dir)

# define optimizer, lr not important at this point
optimizer0 = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=conf.wd)


best_state = {'epoch':-1, 'val_acc':0, 'val_auc':0, 'val_f1':0, 'test_acc':0, 'test_auc':0, 'test_f1':0}
train_epoch = conf.train_epoch
for epoch in range(train_epoch):
    train_one_epoch_multitask(model, criterion, train_loader, optimizer0, device, epoch, conf)


    val_auc, val_acc, val_f1, val_loss = evaluate_multitask(model, criterion, val_loader, device, conf, 'Val')
    test_auc, test_acc, test_f1, test_loss = evaluate_multitask(model, criterion, test_loader, device, conf, 'Test')
    #tma_auc, tma_acc, tma_f1, tma_loss = evaluate_multitask(model, criterion, tma_loader, device, conf, 'TMA')

    if conf.wandb_mode != 'disabled':
        wandb.log({'perf/val_acc1': val_acc}, commit=False)
        wandb.log({'perf/val_auc': val_auc}, commit=False)
        wandb.log({'perf/val_f1': val_f1}, commit=False)
        wandb.log({'perf/val_loss': val_loss}, commit=False)
        wandb.log({'perf/test_acc1': test_acc}, commit=False)
        wandb.log({'perf/test_auc': test_auc}, commit=False)
        wandb.log({'perf/test_f1': test_f1}, commit=False)
        wandb.log({'perf/test_loss': test_loss}, commit=False)


    if val_f1 + val_auc > best_state['val_f1'] + best_state['val_auc']:
        best_state['epoch'] = epoch
        best_state['val_auc'] = val_auc
        best_state['val_acc'] = val_acc
        best_state['val_f1'] = val_f1
        best_state['test_auc'] = test_auc
        best_state['test_acc'] = test_acc
        best_state['test_f1'] = test_f1
        # best_state['tma_auc'] = tma_auc
        # best_state['tma_acc'] = tma_acc
        # best_state['tma_f1'] = tma_f1
        save_model(conf=conf, model=model, optimizer=optimizer0, epoch=epoch,
            save_path=os.path.join(ckpt_dir, 'checkpoint-best.pth'))
    print('\n')


    save_model(conf=conf, model=model, optimizer=optimizer0, epoch=epoch,
        save_path=os.path.join(ckpt_dir + 'checkpoint_' + 'epoch' + str(epoch) + '.pth'))
print("Results on best epoch:")
print(best_state)
wandb.finish()

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//saved_model/MT/' already exists.
Epoch: [0]  [  0/150]  eta: 0:01:12  lr: 0.000100  sub_loss: 0.0169 (0.0169)  diff_loss: 0.9997 (0.9997)  slide_loss: 0.0216 (0.0216)  time: 0.4832  data: 0.0010  max mem: 26
Epoch: [0]  [100/150]  eta: 0:00:01  lr: 0.000100  sub_loss: 0.0019 (0.0095)  diff_loss: 0.9804 (0.9931)  slide_loss: 0.0002 (0.0061)  time: 0.0262  data: 0.0015  max mem: 900
Epoch: [0]  [149/150]  eta: 0:00:00  lr: 0.000100  sub_loss: 0.0016 (0.0133)  diff_loss: 0.7960 (0.9515)  slide_loss: 0.0003 (0.0144)  time: 0.0222  data: 0.0017  max mem: 900
Epoch: [0] Total time: 0:00:04 (0.0272 s / it)
Val  [0/8]  eta: 0:00:00  loss: 0.0323 (0.0323)  div_loss: -48.6221 (-48.6221)  acc1: 100.0000 (100.0000)  time: 0.0047  data: 0.0007  max mem: 900
Val  [7/8]  eta: 0:00:00  loss: 0.0698 (0.1295)  div_loss: -32.0755 (-32.4129)  acc1: 85.7143 (80.3571)  time: 

In [97]:
# define network
if arch == 'ga':
    model2 = ACMIL_GA(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop= conf.mask_drop)
elif arch == 'ga_mt':
    model2 = ACMIL_GA_MultiTask(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop= conf.mask_drop, n_task = conf.n_task)
else:
    model2 = ACMIL_MHA(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop=conf.mask_drop)
model2.to(device)

# Load the checkpoint
#checkpoint = torch.load(ckpt_dir + 'checkpoint-best.pth')
checkpoint = torch.load(ckpt_dir + 'checkpoint_epoch99.pth')

# Load the state_dict into the model
model2.load_state_dict(checkpoint['model'])


y_pred_tasks_test, y_predprob_task_test, y_true_task_test = predict(model2, criterion, test_loader, device, conf, 'Test')
pred_df_list = []
perf_df_list = []
for i in range(conf.n_task):
    pred_df, perf_df = get_performance(y_predprob_task_test[i], y_true_task_test[i], test_ids, ALL_LABELS[i])
    pred_df_list.append(pred_df)
    perf_df_list.append(perf_df)

all_perd_df = pd.concat(pred_df_list)
all_perf_df = pd.concat(perf_df_list)
print(all_perf_df)

all_perd_df.to_csv(outdir4 + "/n_token" + str(conf.n_token) + "_TEST_pred_df.csv",index = False)
all_perf_df.to_csv(outdir5 + "/n_token" + str(conf.n_token) + "_TEST_perf.csv",index = True)

Test  [ 0/45]  eta: 0:00:00    time: 0.0041  data: 0.0008  max mem: 2370
Test  [44/45]  eta: 0:00:00    time: 0.0042  data: 0.0006  max mem: 2373
Test Total time: 0:00:00 (0.0044 s / it)
               AUC   ACC    F1    F2    F3  Recall  Precision  Specificity  \
SAMPLE_LEVEL  0.66  0.73  0.14  0.17  0.19    0.20       0.11         0.80   
SAMPLE_LEVEL  0.86  0.84  0.53  0.56  0.56    0.57       0.50         0.89   
SAMPLE_LEVEL  0.64  0.71  0.24  0.24  0.25    0.25       0.22         0.81   
SAMPLE_LEVEL  0.60  0.73  0.14  0.19  0.22    0.25       0.10         0.78   
SAMPLE_LEVEL  0.58  0.67  0.44  0.37  0.35    0.33       0.67         0.89   
SAMPLE_LEVEL  0.88  0.87  0.63  0.68  0.69    0.71       0.56         0.89   
SAMPLE_LEVEL  0.89  0.82  0.50  0.54  0.56    0.57       0.44         0.87   

                PR_AUC                                           OUTCOME  
SAMPLE_LEVEL  0.192778                                                AR  
SAMPLE_LEVEL  0.617816  MMR (MSH2, MSH

In [98]:
print(round(all_perf_df['AUC'].mean(),2))

0.73


In [99]:
#PLOT:
pred_df = all_perd_df
SELECTED_LABEL = ['MSI_POS']
#Get True Postives
true_postive_ids = {}
for label in SELECTED_LABEL:
    cond = (pred_df['Y_True'] == pred_df['Pred_Class']) & (pred_df['Y_True'] == 1) & (pred_df['OUTCOME'] == label)
    cur_pred_df = pred_df.loc[cond]
    cur_ids = list(cur_pred_df['SAMPLE_IDs'])
    true_postive_ids[label] = cur_ids

#Get true nagative
true_negative_ids = {}
for label in SELECTED_LABEL:
    cond = (pred_df['Y_True'] == pred_df['Pred_Class']) & (pred_df['Y_True'] == 0) & (pred_df['OUTCOME'] == label)
    cur_pred_df = pred_df.loc[cond]
    cur_ids = list(cur_pred_df['SAMPLE_IDs'])
    true_negative_ids[label] = cur_ids

In [100]:
true_postive_ids

{'MSI_POS': ['OPX_075', 'OPX_107', 'OPX_210', 'OPX_211']}

In [101]:
####################################################################################
#Atention scores
####################################################################################
save_image_size = 250
pixel_overlap = 0
mag_extract = 20
limit_bounds = True
TOP_K = 5
pretrain_model_name = "retccl"
mag_target_prob = 2.5
smooth = True
mag_target_tiss = 1.25

def get_attention_and_tileinfo(pt_label_df, patient_att_score):    
    #Get label
    pt_label_df.reset_index(drop = True, inplace = True)

    #Get attention
    cur_att  = pd.DataFrame({'ATT':list(minmax_normalize(patient_att_score))})
    cur_att.reset_index(drop = True, inplace = True)

    #Comb
    cur_att_df = pd.concat([pt_label_df,cur_att], axis = 1)
    cur_att_df.reset_index(drop = True, inplace = True)

    return cur_att_df

In [106]:
attn[label_index].shape

torch.Size([1, 3, 96])

In [103]:
selected_ids = true_postive_ids[SELECTED_LABEL[0]]
wsi_path = proj_dir + '/data/OPX/'
branches = 1

for pt in selected_ids:
    i = test_ids.index(pt)
    pt = test_ids[i]
    print(pt)

    save_location = outdir4 + SELECTED_LABEL[0] + "/"
    save_location =  save_location  + pt + "/"
    create_dir_if_not_exists(save_location)
    
    _file = wsi_path + pt + ".tif"
    oslide = openslide.OpenSlide(_file)
    save_name = str(Path(os.path.basename(_file)).with_suffix(''))
    
    first_batch = list(test_loader)[i]
    feat = first_batch[0].to(device)
    sub_preds, slide_preds, attn = model(feat)
    #cur_pt_att =  attn[0,:,:].mean(0).cpu().detach().numpy() #Take the mean across branches without softmax
    label_index = ALL_LABELS.index(SELECTED_LABEL[0])
    cur_pt_att = attn[label_index][branches].mean(0).cpu().detach().numpy()
    #branches = 0
    #cur_pt_att = torch.softmax(attn, dim=-1)[0][branches].cpu().detach().numpy() 
    
    #Get all tile info include noncancer tile
    alltileinfo_dir = proj_dir + 'intermediate_data/TODELETE_cancer_prediction_results110224/' + "IMSIZE250_OL0" + "/"
    tile_info_df = pd.read_csv(alltileinfo_dir + pt + "/"  + save_name + "_tiles.csv")
    cur_pt_info = test_info[i]
    #Combine current pt_info an all tile info
    #cur_pt_info = tile_info_df.merge(cur_pt_info, on = list(tile_info_df.columns), how = "left")
    
    cur_att_df = get_attention_and_tileinfo(cur_pt_info, cur_pt_att)
    #cur_att_df.loc[pd.isna(cur_att_df['ATT']),'ATT'] = 0.0001
    
    #Generate tiles
    tiles, tile_lvls, physSize, base_mag = generate_deepzoom_tiles(oslide,save_image_size, pixel_overlap, limit_bounds)
    
    #get level 0 size in px
    l0_w = oslide.level_dimensions[0][0]
    l0_h = oslide.level_dimensions[0][1]
    
    #1.25x tissue detection for mask
    from Utils import get_downsample_factor, get_image_at_target_mag
    from Utils import do_mask_original,check_tissue,whitespace_check
    import cv2
    if 'OPX' in pt:
        rad_tissue = 5
    elif '(2017-0133)' in pt:
        rad_tissue = 2
    lvl_resize_tissue = get_downsample_factor(base_mag,target_magnification = mag_target_tiss) #downsample factor
    lvl_img = get_image_at_target_mag(oslide,l0_w, l0_h,lvl_resize_tissue)
    tissue, he_mask = do_mask_original(lvl_img, lvl_resize_tissue, rad = rad_tissue)
    
    #2.5x for probability maps
    lvl_resize = get_downsample_factor(base_mag,target_magnification = mag_target_prob) #downsample factor
    x_map = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
    x_count = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
    
    
    for index, row in cur_att_df.iterrows():
        cur_xy = row['TILE_XY_INDEXES'].strip("()").split(", ")
        x ,y = int(cur_xy[0]) , int(cur_xy[1])
        
        #Extract tile for prediction
        lvl_in_deepzoom = tile_lvls.index(mag_extract)
        tile_starts, tile_ends, save_coords, tile_coords = extract_tile_start_end_coords(tiles, lvl_in_deepzoom, x, y) #get tile coords
        map_xstart, map_xend, map_ystart, map_yend = get_map_startend(tile_starts,tile_ends,lvl_resize) #Get current tile position in map
    
        #Store predicted probabily in map and count
        try: 
            x_count[map_xstart:map_xend,map_ystart:map_yend] += 1
            x_map[map_xstart:map_xend,map_ystart:map_yend] += row['ATT']
        except:
            pass
    
    print('post-processing')
    x_count = np.where(x_count < 1, 1, x_count)
    x_map = x_map / x_count
    x_map[x_map>1]=1
    
    #Get the following before smooth
    he_mask = cv2.resize(np.uint8(he_mask),(x_map.shape[1],x_map.shape[0])) #resize to output image size
    cond1 = he_mask < 1 #Background
    cond2 = (he_mask == 1) & (x_map == 0) #is tissue, but not selected
    smooth = True
    
    if smooth == True:
        #x_sm = filters.gaussian(x_map, sigma=0)
        x_sm = np.where(x_map != 0, filters.gaussian(x_map, sigma=10), x_map)
    if smooth == False:
        x_sm = x_map
    
    #TODO:
    #get cancer_mask:
    # cancer_mask == 
    # x_sm[(he_mask == 1) & (x_sm == 0)] = 0.1 #If tissue map value > 1, then x_sm = 1
    x_sm[cond1] = 0 #Background
    x_sm[cond2] = 0.1 #Is tissue, but not selected 
    
    # Define the colors for the sequential colormap (black to fluorescent green)
    colors = ["#4B0082", "#39FF14"]  # Black to Fluorescent Green
    # Create the sequential colormap
    cmap_name = "black_to_fluorescent_green"
    from matplotlib.colors import LinearSegmentedColormap
    from matplotlib.colors import ListedColormap
    sequential_cmap = LinearSegmentedColormap.from_list(cmap_name, colors)
    cmap =  plt.cm.Spectral_r #sequential_cmap # plt.cm.YlGn_r
    cmap_colors = cmap(np.arange(cmap.N))
    cmap_colors[0] = np.array([0.95, 0.95, 0.95, 1]) #np.array([1, 1, 1, 1])  # Set the first color (corresponding to 0) to white
    cmap_colors[1] = np.array([0, 0, 0.545, 1])  # RGB for dark blue
    custom_cmap = ListedColormap(cmap_colors)
    
    plt.imshow(x_sm, cmap=custom_cmap) #Spectral_r
    plt.colorbar()
    plt.savefig(os.path.join(save_location, save_name + '_attention.png'), dpi=500,bbox_inches='tight')
    plt.show()
    plt.close()


    #Top attented tiles
    save_location2 = save_location + "top_tiles/"
    create_dir_if_not_exists(save_location2)
    
    #Get a Attention, and corresponding tiles
    cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = False) 
    cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
            
    for i in range(TOP_K):
        cur_pulled_img = cur_pulled_img_obj[i][0] #image
        cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
        cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
        coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
        tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
        cur_pulled_img.save(os.path.join(save_location2, tile_save_name))
    
    #Bot attented tiles
    save_location2 = save_location + "bot_tiles/"
    create_dir_if_not_exists(save_location2)
    
    #Get a Attention, and corresponding tiles
    cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = True) 
    cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
    
    for i in range(TOP_K):
        cur_pulled_img = cur_pulled_img_obj[i][0] #image
        cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
        cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
        coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
        tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
        cur_pulled_img.save(os.path.join(save_location2, tile_save_name))

OPX_075
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//predictions/MSI_POS/OPX_075/' already exists.


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [79]:
y_pred_tasks_test, y_predprob_task_test, y_true_task_test = predict(model, criterion, tma_loader, device, conf, 'TMA')
pred_df_list = []
perf_df_list = []
for i in range(conf.n_task):
    if i not in [1,5,6]:
        pred_df, perf_df = get_performance(y_predprob_task_test[i], y_true_task_test[i], tma_ids, ALL_LABELS[i])
        pred_df_list.append(pred_df)
        perf_df_list.append(perf_df)

all_perd_df = pd.concat(pred_df_list)
all_perf_df = pd.concat(perf_df_list)
print(all_perf_df)
all_perd_df.to_csv(outdir4 + "/n_token" + str(conf.n_token) + "_TMA_pred_df.csv",index = False)
all_perf_df.to_csv(outdir5 + "/n_token" + str(conf.n_token) + "_TMA_perf.csv",index = True)

TMA  [  0/355]  eta: 0:00:02    time: 0.0059  data: 0.0011  max mem: 2370
TMA  [100/355]  eta: 0:00:00    time: 0.0022  data: 0.0001  max mem: 2370
TMA  [200/355]  eta: 0:00:00    time: 0.0133  data: 0.0001  max mem: 2370
TMA  [300/355]  eta: 0:00:00    time: 0.0021  data: 0.0001  max mem: 2370
TMA  [354/355]  eta: 0:00:00    time: 0.0021  data: 0.0001  max mem: 2370
TMA Total time: 0:00:01 (0.0029 s / it)
               AUC   ACC    F1    F2    F3  Recall  Precision  Specificity  \
SAMPLE_LEVEL  0.62  0.49  0.38  0.29  0.27    0.25       0.75         0.87   
SAMPLE_LEVEL  0.51  0.63  0.31  0.27  0.25    0.24       0.41         0.82   
SAMPLE_LEVEL  0.61  0.70  0.39  0.34  0.33    0.32       0.50         0.86   
SAMPLE_LEVEL  0.55  0.54  0.33  0.26  0.25    0.23       0.60         0.85   

                PR_AUC OUTCOME  
SAMPLE_LEVEL  0.709850      AR  
SAMPLE_LEVEL  0.393212    PTEN  
SAMPLE_LEVEL  0.442043     RB1  
SAMPLE_LEVEL  0.545806    TP53  


In [58]:
print(round(all_perf_df['AUC'].mean(),2))

0.57


In [90]:
#PLOT:
SELECTED_LABEL = ['TP53']
pred_df = all_perd_df
#Get True Postives
true_postive_ids = {}
for label in SELECTED_LABEL:
    cond = (pred_df['Y_True'] == pred_df['Pred_Class']) & (pred_df['Y_True'] == 1) & (pred_df['OUTCOME'] == label)
    cur_pred_df = pred_df.loc[cond]
    cur_ids = list(cur_pred_df['SAMPLE_IDs'])
    true_postive_ids[label] = cur_ids

#Get true nagative
true_negative_ids = {}
for label in SELECTED_LABEL:
    cond = (pred_df['Y_True'] == pred_df['Pred_Class']) & (pred_df['Y_True'] == 0) & (pred_df['OUTCOME'] == label)
    cur_pred_df = pred_df.loc[cond]
    cur_ids = list(cur_pred_df['SAMPLE_IDs'])
    true_negative_ids[label] = cur_ids

print(true_postive_ids)

{'TP53': ['TMA97A-1-2', 'TMA97A-1-9', 'TMA97A-2-1', 'TMA97A-2-4', 'TMA97A-3-12', 'TMA97A-4-10', 'TMA97A-4-11', 'TMA97A-4-2', 'TMA97A-4-3', 'TMA97A-4-4', 'TMA97A-4-5', 'TMA97A-4-6', 'TMA97A-4-8', 'TMA97A-4-9', 'TMA97A-5-1', 'TMA97B-4-3', 'TMA97B-4-4', 'TMA97B-4-6', 'TMA97B-4-8', 'TMA97B-7-1', 'TMA97B-7-11', 'TMA97B-7-4', 'TMA97B-7-8', 'TMA97B-8-3', 'TMA97B-8-5', 'TMA97C-4-3', 'TMA97D-1-3', 'TMA97D-5-1', 'TMA97D-6-3', 'TMA97F-6-8', 'TMA97F-6-9', 'TMA97G-2-11', 'TMA97G-2-9', 'TMA97H-2-10', 'TMA97H-3-11', 'TMA97H-3-8', 'TMA97H-4-3', 'TMA97H-4-4', 'TMA97H-5-11', 'TMA97H-5-12', 'TMA97H-5-9']}


In [91]:
####################################################################################
#Atention scores
####################################################################################
save_image_size = 250
pixel_overlap = 0
mag_extract = 20
limit_bounds = True
TOP_K = 5
pretrain_model_name = "retccl"
mag_target_prob = 2.5
smooth = True
mag_target_tiss = 1.25

def get_attention_and_tileinfo(pt_label_df, patient_att_score):    
    #Get label
    pt_label_df.reset_index(drop = True, inplace = True)

    #Get attention
    cur_att  = pd.DataFrame({'ATT':list(minmax_normalize(patient_att_score))})
    cur_att.reset_index(drop = True, inplace = True)

    #Comb
    cur_att_df = pd.concat([pt_label_df,cur_att], axis = 1)
    cur_att_df.reset_index(drop = True, inplace = True)

    return cur_att_df

In [94]:
selected_ids = true_postive_ids[SELECTED_LABEL[0]]
wsi_path = proj_dir + '/data/OPX/'

for pt in selected_ids:
    i = tma_ids.index(pt)
    pt = tma_ids[i]
    print(pt)

    save_location = outdir4 + SELECTED_LABEL[0] + "/"
    save_location =  save_location  + pt + "/"
    create_dir_if_not_exists(save_location)
    
    _file = wsi_path + pt + ".tif"
    oslide = openslide.OpenSlide(_file)
    save_name = str(Path(os.path.basename(_file)).with_suffix(''))
    
    first_batch = list(tma_loader)[i]
    feat = first_batch[0].to(device)
    sub_preds, slide_preds, attn = model(feat)
    #cur_pt_att =  attn[0,:,:].mean(0).cpu().detach().numpy() #Take the mean across branches without softmax
    label_index = ALL_LABELS.index(SELECTED_LABEL[0])
    branches = 0
    cur_pt_att = attn[label_index][branches].mean(0).cpu().detach().numpy()
    #branches = 0
    #cur_pt_att = torch.softmax(attn, dim=-1)[0][branches].cpu().detach().numpy() 
    
    #Get all tile info include noncancer tile
    alltileinfo_dir = proj_dir + 'intermediate_data/1_tile_pulling_and_tissue_detect/TAN_TMA_Cores' + "IMSIZE250_OL0" + "/"
    tile_info_df = pd.read_csv(alltileinfo_dir + pt + "/"  + save_name + "_tiles.csv")
    cur_pt_info = tma_info[i]
    #Combine current pt_info an all tile info
    #cur_pt_info = tile_info_df.merge(cur_pt_info, on = list(tile_info_df.columns), how = "left")
    
    cur_att_df = get_attention_and_tileinfo(cur_pt_info, cur_pt_att)
    #cur_att_df.loc[pd.isna(cur_att_df['ATT']),'ATT'] = 0.0001
    
    #Generate tiles
    tiles, tile_lvls, physSize, base_mag = generate_deepzoom_tiles(oslide,save_image_size, pixel_overlap, limit_bounds)
    
    #get level 0 size in px
    l0_w = oslide.level_dimensions[0][0]
    l0_h = oslide.level_dimensions[0][1]
    
    #1.25x tissue detection for mask
    from Utils import get_downsample_factor, get_image_at_target_mag
    from Utils import do_mask_original,check_tissue,whitespace_check
    import cv2
    if 'OPX' in pt:
        rad_tissue = 5
    elif '(2017-0133)' in pt:
        rad_tissue = 2
    lvl_resize_tissue = get_downsample_factor(base_mag,target_magnification = mag_target_tiss) #downsample factor
    lvl_img = get_image_at_target_mag(oslide,l0_w, l0_h,lvl_resize_tissue)
    tissue, he_mask = do_mask_original(lvl_img, lvl_resize_tissue, rad = rad_tissue)
    
    #2.5x for probability maps
    lvl_resize = get_downsample_factor(base_mag,target_magnification = mag_target_prob) #downsample factor
    x_map = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
    x_count = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
    
    
    for index, row in cur_att_df.iterrows():
        cur_xy = row['TILE_XY_INDEXES'].strip("()").split(", ")
        x ,y = int(cur_xy[0]) , int(cur_xy[1])
        
        #Extract tile for prediction
        lvl_in_deepzoom = tile_lvls.index(mag_extract)
        tile_starts, tile_ends, save_coords, tile_coords = extract_tile_start_end_coords(tiles, lvl_in_deepzoom, x, y) #get tile coords
        map_xstart, map_xend, map_ystart, map_yend = get_map_startend(tile_starts,tile_ends,lvl_resize) #Get current tile position in map
    
        #Store predicted probabily in map and count
        try: 
            x_count[map_xstart:map_xend,map_ystart:map_yend] += 1
            x_map[map_xstart:map_xend,map_ystart:map_yend] += row['ATT']
        except:
            pass
    
    print('post-processing')
    x_count = np.where(x_count < 1, 1, x_count)
    x_map = x_map / x_count
    x_map[x_map>1]=1
    
    #Get the following before smooth
    he_mask = cv2.resize(np.uint8(he_mask),(x_map.shape[1],x_map.shape[0])) #resize to output image size
    cond1 = he_mask < 1 #Background
    cond2 = (he_mask == 1) & (x_map == 0) #is tissue, but not selected
    smooth = True
    
    if smooth == True:
        #x_sm = filters.gaussian(x_map, sigma=0)
        x_sm = np.where(x_map != 0, filters.gaussian(x_map, sigma=10), x_map)
    if smooth == False:
        x_sm = x_map
    
    #TODO:
    #get cancer_mask:
    # cancer_mask == 
    # x_sm[(he_mask == 1) & (x_sm == 0)] = 0.1 #If tissue map value > 1, then x_sm = 1
    x_sm[cond1] = 0 #Background
    x_sm[cond2] = 0.1 #Is tissue, but not selected 
    
    # Define the colors for the sequential colormap (black to fluorescent green)
    colors = ["#4B0082", "#39FF14"]  # Black to Fluorescent Green
    # Create the sequential colormap
    cmap_name = "black_to_fluorescent_green"
    from matplotlib.colors import LinearSegmentedColormap
    from matplotlib.colors import ListedColormap
    sequential_cmap = LinearSegmentedColormap.from_list(cmap_name, colors)
    cmap =  plt.cm.Spectral_r #sequential_cmap # plt.cm.YlGn_r
    cmap_colors = cmap(np.arange(cmap.N))
    cmap_colors[0] = np.array([0.95, 0.95, 0.95, 1]) #np.array([1, 1, 1, 1])  # Set the first color (corresponding to 0) to white
    cmap_colors[1] = np.array([0, 0, 0.545, 1])  # RGB for dark blue
    custom_cmap = ListedColormap(cmap_colors)
    
    plt.imshow(x_sm, cmap=custom_cmap) #Spectral_r
    plt.colorbar()
    plt.savefig(os.path.join(save_location, save_name + '_attention.png'), dpi=500,bbox_inches='tight')
    plt.show()
    plt.close()


    #Top attented tiles
    save_location2 = save_location + "top_tiles/"
    create_dir_if_not_exists(save_location2)
    
    #Get a Attention, and corresponding tiles
    cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = False) 
    cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
            
    for i in range(TOP_K):
        cur_pulled_img = cur_pulled_img_obj[i][0] #image
        cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
        cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
        coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
        tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
        cur_pulled_img.save(os.path.join(save_location2, tile_save_name))
    
    #Bot attented tiles
    save_location2 = save_location + "bot_tiles/"
    create_dir_if_not_exists(save_location2)
    
    #Get a Attention, and corresponding tiles
    cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = True) 
    cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
    
    for i in range(TOP_K):
        cur_pulled_img = cur_pulled_img_obj[i][0] #image
        cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
        cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
        coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
        tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
        cur_pulled_img.save(os.path.join(save_location2, tile_save_name))

TMA97A-1-2
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out02122025/uni2/TrainOL100_TestOL0_TFT0.9/FOLD0/MT//predictions/TP53/TMA97A-1-2/' created.


OpenSlideUnsupportedFormatError: Unsupported or missing image file

In [89]:
true_postive_ids

{'TP53': ['TMA97A-1-2',
  'TMA97A-1-9',
  'TMA97A-2-1',
  'TMA97A-2-4',
  'TMA97A-3-12',
  'TMA97A-4-10',
  'TMA97A-4-11',
  'TMA97A-4-2',
  'TMA97A-4-3',
  'TMA97A-4-4',
  'TMA97A-4-5',
  'TMA97A-4-6',
  'TMA97A-4-8',
  'TMA97A-4-9',
  'TMA97A-5-1',
  'TMA97B-4-3',
  'TMA97B-4-4',
  'TMA97B-4-6',
  'TMA97B-4-8',
  'TMA97B-7-1',
  'TMA97B-7-11',
  'TMA97B-7-4',
  'TMA97B-7-8',
  'TMA97B-8-3',
  'TMA97B-8-5',
  'TMA97C-4-3',
  'TMA97D-1-3',
  'TMA97D-5-1',
  'TMA97D-6-3',
  'TMA97F-6-8',
  'TMA97F-6-9',
  'TMA97G-2-11',
  'TMA97G-2-9',
  'TMA97H-2-10',
  'TMA97H-3-11',
  'TMA97H-3-8',
  'TMA97H-4-3',
  'TMA97H-4-4',
  'TMA97H-5-11',
  'TMA97H-5-12',
  'TMA97H-5-9']}